# Hypothesis Testing: Correlation

In [ ]:
%matplotlib inline
from math import sqrt

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

import correlation_lesson_util

plt.ion()
plt.rc('figure', figsize=(13, 7))

## Hypothesis Testing Review / Recap

### Limits of Hypothesis Testing

- Hypothesis provides statistical evidence to tell us whether the results we are seeing are due to chance
- Very important for, e.g., RCTs w/ a small n
- With a large n, almost everything is statistically significant
- Provides guidance for borderline cases
- Doesn't say anything about effect size
- **Never** do stats testing in isolation, it's a supporting piece of evidence, but must be combined w/ other analysis and viz to support business intelligence

### Hypothesis Testing Simulation Based Demo

- What is a probability? A frequency
- probabilities tell us what % of possible universes contain that outcome
- what's a p-value? P(data | H0) (or more extreme data)
- what are simulations? a way to calculate probability
- simulations represent one universe in the multiverse

Let's say we have 2 groups: virtual students and in-person students and we want to examine their scores on the stortelling project. If it truly is the case that there is no difference between these groups, how likely is it that we observe one?

In [ ]:
df = correlation_lesson_util.viz_ttest_h0_is_true()
df.head()

In [ ]:
n_sims = 1000
simulation_results = []

for _ in range(n_sims):
    sample = df.sample(25)
    virtual_students = sample[sample.student_type == 'Virtual']
    in_person_students = sample[sample.student_type == 'In-Person']
    h_test_results = stats.ttest_ind(virtual_students.grade, in_person_students.grade)
    simulation_results.append(h_test_results)

(pd.DataFrame(simulation_results).pvalue < .05).mean()

In [ ]:
df = correlation_lesson_util.viz_ttest_h0_is_false()
df.head()

In [ ]:
n_sims = 1000
simulation_results = []

for _ in range(n_sims):
    sample = df.sample(25)
    virtual_students = sample[sample.student_type == 'Virtual']
    in_person_students = sample[sample.student_type == 'In-Person']
    h_test_results = stats.ttest_ind(virtual_students.grade, in_person_students.grade)
    simulation_results.append(h_test_results)

(pd.DataFrame(simulation_results).pvalue < .05).mean()

https://xkcd.com/882/

## Comparing 2 Continuous Variables: Correlation

- Pearson's correlation coefficient or **Pearson's r** is the test statistic we get from a correlation test
- For discrete or non-normal data: Spearman's rank correlation
- Compares 2 continuous variables
- Unitless; tells us how much two variables tend to move together
- $H_0$: there is no linear relationship between the two variables

In [ ]:
url = "https://gist.githubusercontent.com/ryanorsinger/2c13a71421037af127e9fa7fa1463cad/raw/3eb443414078b51af33fdb2d211159e5f3e220ab/exam_scores.csv"
df = pd.read_csv(url)
df.head()

## Example Test

What's the relationship between exam score and hours studied?

In [ ]:
df.plot.scatter(y='exam_score', x='hours_studied')

Abbreviated Hypothesis Testing Procedure:

1. Setup
2. Calculate test statistic + p-value
3. Conclude

- $\alpha = .05$
- $H_0$: There is no linear relationship between exam score and number of hours studied.
- $H_a$: There is a linear relationship between exam score and number of hours studied.

In [ ]:
x = df.hours_studied
y = df.exam_score

In [ ]:
r, p = stats.pearsonr(x, y)
print('r =', r)
print('p =', p)

Conclusion: we reject the null hypothesis that there is no linear relationship between exam score and number of hours studied.

## Examples of Different Correlations

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]

plt.plot(x, y, marker='o')
plt.title('Perfect positive correlation, (r=1)')

In [ ]:
x = [3, 2, 1]
y = [1, 2, 3]

plt.plot(x, y, marker='o')
plt.title('Perfect negative correlation (r=-1)')

In [ ]:
x = [1, 2, 3]
y = [1, 1, 1]

plt.plot(x, y, marker='o')
plt.title('Absolutely no correlation (r=0)')

In [ ]:
x = np.random.randn(100)
y = np.random.randn(100)

r, p = stats.pearsonr(x, y)

plt.scatter(x, y)
plt.title(r'no correlation, ($r \approx 0$)', size=16)
plt.text(plt.xlim()[-1] - .25, plt.ylim()[-1] - .25, f'{r=:.4}\n{p=:.4}', ha='right', va='top')

In [ ]:
x = np.random.randn(100)
y = x + (np.random.randn(100) * 1.1)

r, p = stats.pearsonr(x, y)

plt.scatter(x, y)
plt.title('Positive Correlation')
plt.text(plt.xlim()[-1] - .25, plt.ylim()[-1] - .25, f'{r=:.4}\n{p=:.4f}', ha='right', va='top')

In [ ]:
x = np.random.randn(100)
y = -x + (np.random.randn(100) * 1.1)

r, p = stats.pearsonr(x, y)

plt.scatter(x, y)
plt.title('Negative Correlation')
plt.text(plt.xlim()[-1] - .25, plt.ylim()[-1] - .25, f'{r=:.4}\n{p=:.4f}', ha='right', va='top')

## Correlation "gotchas"

When looking at correlation, keep in mind:

1. [Correlation is not causation!](https://xkcd.com/552/) It could be something else or alltogether spurious.
1. This is just looking at a linear relationships
1. Correlation itself just looks 2 variables, there could be more going on
1. Correlation doesn't tell us how large the relationship is

### Example: non-linear relationship

In [ ]:
x = np.linspace(-3, 3)
y = x ** 2

r, p = stats.pearsonr(x, y)

plt.scatter(x, y)
plt.text(0, 8, f'{r=:.4f}\n{p=:.4f}')

### Example: confounding variable

In [ ]:
df.plot.scatter(y='exam_score', x='coffee_consumed')

r, p = stats.pearsonr(df.exam_score, df.coffee_consumed)
print('r=', r)

In [ ]:
r, p = stats.pearsonr(df.coffee_consumed, df.hours_slept)
df.plot.scatter(y='coffee_consumed', x='hours_slept')
print('p=', p)
print('r=', r)

### Example: relationship magnitude

In [ ]:
coffee_consumption = [0, 1, 2, 3, 4]
exam_score = [80, 80.1, 80.2, 80.3, 80.4]

r, p = stats.pearsonr(coffee_consumption, exam_score)

plt.plot(coffee_consumption, exam_score, marker='o')
plt.ylim(0, 100)
print('r=', r)

## Exercise Guidance

What does it mean to "control for"?

In [ ]:
df.head()

In [ ]:
df.plot.scatter(y='exam_score', x='hours_studied')

What if want to control for handedness?

In [ ]:
sns.relplot(data=df, y='exam_score', x='hours_studied', col='handedness')

In [ ]:
lefties = df[df.handedness == 'left']
righties = df[df.handedness == 'right']

In [ ]:
r, p = stats.pearsonr(lefties.exam_score, lefties.hours_studied)
print('Lefties')
print('  r =', r)
print('  p =', p)

In [ ]:
r, p = stats.pearsonr(righties.exam_score, righties.hours_studied)
print('Righties')
print('  r =', r)
print('  p =', p)

## Correlation Simulation Demo

In [ ]:
df = correlation_lesson_util.viz_pearson_h0_is_true()
df.head()

In [ ]:
simulations = []
for _ in range(1000):
    sample = df.sample(20)
    simulations.append(stats.pearsonr(sample.x, sample.y))
(pd.DataFrame(simulations, columns=['r', 'p']).sort_values(by='p').p < .05).mean()

In [ ]:
df = correlation_lesson_util.viz_pearson_h0_is_false()
df.head()

In [ ]:
simulations = []
for _ in range(1000):
    sample = df.sample(20)
    simulations.append(stats.pearsonr(sample.x, sample.y))
(pd.DataFrame(simulations, columns=['r', 'p']).sort_values(by='p').p < .05).mean()